<a href="https://colab.research.google.com/github/simecek/dspracticum2021/blob/main/lesson04/Transfer_Learning_DogsAndCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical

## Data

In [2]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
EPOCHS = 5
IMG_SIZE = (160, 160)
IMG_SHAPE = IMG_SIZE + (3,)

68616192/68606236 [==============================] - 1s 0us/step


## MobileNetV2

In [3]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 80, 80, 32)   0           bn_Conv1[0][0]                   
________________

## Transfer Learning

### Model

In [4]:
for layer in base_model.layers:
    layer.trainable = False
    
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(units=2, activation="softmax")(x)

model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Data Generators

In [5]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=32
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=32
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Training

In [6]:
history = model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=validation_generator)

Epoch 1/5
63/63 [==============================] - 55s 375ms/step - loss: 0.2757 - accuracy: 0.8755 - val_loss: 0.1068 - val_accuracy: 0.9620
Epoch 2/5
63/63 [==============================] - 22s 348ms/step - loss: 0.1195 - accuracy: 0.9540 - val_loss: 0.1038 - val_accuracy: 0.9560
Epoch 3/5
63/63 [==============================] - 22s 355ms/step - loss: 0.1023 - accuracy: 0.9590 - val_loss: 0.0871 - val_accuracy: 0.9710
Epoch 4/5
63/63 [==============================] - 23s 361ms/step - loss: 0.1002 - accuracy: 0.9585 - val_loss: 0.1064 - val_accuracy: 0.9540
Epoch 5/5
63/63 [==============================] - 23s 363ms/step - loss: 0.0836 - accuracy: 0.9680 - val_loss: 0.0868 - val_accuracy: 0.9700


## Export to TFjs

In [8]:
!pip install tensorflowjs

     |████████████████████████████████| 64 kB 1.9 MB/s 


In [9]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "./export_model")

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [10]:
import shutil
shutil.make_archive("export_model", 'zip', "export_model/")

from google.colab import files
files.download('export_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

And finally...

1.   Copy the downloaded file into `docs` folder of your GitHub repository and unzip it into `export_model` folder
2.   Add `index.html` and `index.js` from https://github.com/simecek/dspracticum2021/tree/main/docs
3.   Change the path in `index.js` from `https://raw.githubusercontent.com/simecek/dspracticum2021/master/docs/export_model/model.json` to the path in your repo
4. Change the setting of your repository to use GitHub pages (`main` branch, `docs` folder)

Do not forget to push from local to GitHub.


, 